# Lab 4, Exercise 1

In [1]:
import numpy as np
import pandas as pd

## Load data 

Here is an example of the first couple rows from the data:

| id | dur       | proto | service | state | spkts | dpkts | sbytes | dbytes | rate        | sttl | dttl | sload     | dload | sloss | dloss | sinpkt | dinpkt | sjit | djit | swin | stcpb | dtcpb | dwin | tcprtt | synack | ackdat | smean | dmean | trans\_depth | response\_body\_len | ct\_srv\_src | ct\_state\_ttl | ct\_dst\_ltm | ct\_src\_dport\_ltm | ct\_dst\_sport\_ltm | ct\_dst\_src\_ltm | is\_ftp\_login | ct\_ftp\_cmd | ct\_flw\_http\_mthd | ct\_src\_ltm | ct\_srv\_dst | is\_sm\_ips\_ports | attack\_cat | label |
|----|-----------|-------|---------|-------|-------|-------|--------|--------|-------------|------|------|-----------|-------|-------|-------|--------|--------|------|------|------|-------|-------|------|--------|--------|--------|-------|-------|--------------|---------------------|--------------|----------------|--------------|---------------------|---------------------|-------------------|----------------|--------------|---------------------|--------------|--------------|--------------------|-------------|-------|
| 1  | 0\.000011 | udp   | \-      | INT   | 2     | 0     | 496    | 0      | 90909\.0902 | 254  | 0    | 180363632 | 0     | 0     | 0     | 0\.011 | 0      | 0    | 0    | 0    | 0     | 0     | 0    | 0      | 0      | 0      | 248   | 0     | 0            | 0                   | 2            | 2              | 1            | 1                   | 1                   | 2                 | 0              | 0            | 0                   | 1            | 2            | 0                  | Normal      | 0     |


In [2]:
# Load the training and test data corresponding to exercise 1
# Create two separate pandas dataframes for the training and test data
# For each dataframe, import the following CSV data
# training set: 'data/exercise1/UNSW_NB15_training-set.csv'
# test set: 'data/exercise1/UNSW_NB15_testing-set.csv'

# CODE HERE
train_df = pd.read_csv('data/exercise1/UNSW_NB15_training-set.csv')
test_df = pd.read_csv('data/exercise1/UNSW_NB15_testing-set.csv')

### Part 1 
Keep the two dataframes separate and create train/test data and labels.  This will be used to experiment with the case where there are different types of activities in the training versus test set.

In [4]:
# Do all the following operations in the cell on both the dataframes separately
# 1) Keep only the datapoints where the 'attack_cat' column is equal to either 'Normal' or 'Fuzzers'
# CODE HERE
train_set = train_df.loc[train_df['attack_cat'].isin(['Normal', 'Fuzzers'])]
test_set = test_df.loc[test_df['attack_cat'].isin(['Normal', 'Fuzzers'])]

# 2) Get the labels from the dataframe (i.e., the values in the 'attack_cat' column)
# CODE HERE
train_label = train_set['attack_cat'].tolist()
test_label = test_set['attack_cat'].tolist()

# 3) Keep only the features we care about for this experiment.
# We only care about the numerical features between column 'spkts' and 'is_sm_ips_ports' (inclusive)
# CODE HERE
col_names = list(train_set.columns)
col_headers = col_names[col_names.index('spkts'):col_names.index('is_sm_ips_ports') + 1]
train_set = train_set.filter(items=col_headers)
test_set = test_set.filter(items=col_headers)

# You should now have four inputs usable for scikit-learn:
# training data
# training labels
# test data
# test labels
# Hint: You may have to add some minor code in the above to get the data ready for scikit-learn

### Part 2 
Create a new training/test split by combining the dataframes into one.  Then split the dataframe randomly into train/test data and labels.  This will be used to experiment with the case where there are largely the same types of activities in the training and test set.

In [5]:
# Combine the dataframes into a single dataframe, then do the following
# CODE HERE
combine_df = pd.concat([train_df, test_df])

# 1) Keep only the datapoints where the 'attack_cat' column is equal to either 'Normal' or 'Fuzzers'
# CODE HERE
combine_set = combine_df.loc[combine_df['attack_cat'].isin(['Normal', 'Fuzzers'])]

# 2) Get the labels from the dataframe (i.e., the values in the 'attack_cat' column)
# CODE HERE
combine_label = combine_set['attack_cat'].tolist()

# 3) Keep only features we care about for this experiment.
# We only care about the numerical features between column 'spkts' and 'is_sm_ips_ports' (inclusive)
# CODE HERE
col_names = list(combine_set.columns)
col_headers = col_names[col_names.index('spkts'):col_names.index('is_sm_ips_ports') + 1]
combine_set = combine_set.filter(items=col_headers)

# 4) Create a random split; put 50% of the data into the training set and the other 50% into the test set
# Use scikit-learn's 'train_test_split'
# Hint: You may have to add some minor code in the above to get the data ready for scikit-learn's 'train_test_split'
# CODE HERE
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(combine_set, combine_label, test_size=0.5)

# You should now have four inputs usable for scikit-learn that are distinct from the inputs you created in Part 1.
# The inputs should correspond to:
# training data
# training labels
# test data
# test labels

## Train a classifier

In [8]:
# For each of the train/test splits, create a separate random forest with default sklearn parameters
# Hint: Ignoring import statements, each random forest can be created in a single line of code
# CODE HERE
from sklearn.ensemble import RandomForestClassifier
rnd_forest = RandomForestClassifier().fit(train_set, train_label)
rnd_forest2 = RandomForestClassifier().fit(x_train, y_train)

## Inference and results

In [10]:
# For each of the train/test splits and associated random forest, do the following:
from sklearn.metrics import classification_report
# 1) Predict labels on the training data
# CODE HERE
pred_label = rnd_forest.predict(train_set)
pred_label2 = rnd_forest2.predict(x_train)

# 2) Print metrics on the training data; use sklearn's implementation of precision, recall, f1, and accuracy
# CODE HERE
print('Training data results')
print(classification_report(train_label, pred_label))
print(classification_report(y_train, pred_label2))

# 3) Predict labels on the test data
# CODE HERE
pred_label = rnd_forest.predict(test_set)
pred_label2 = rnd_forest2.predict(x_test)

# 4) Print metrics on the test data; again, use precision, recall, f1, and accuracy
# CODE HERE
print('Test data results')
print(classification_report(test_label, pred_label))
print(classification_report(y_test, pred_label2))

Training data results
              precision    recall  f1-score   support

     Fuzzers       1.00      1.00      1.00      6062
      Normal       1.00      1.00      1.00     37000

    accuracy                           1.00     43062
   macro avg       1.00      1.00      1.00     43062
weighted avg       1.00      1.00      1.00     43062

              precision    recall  f1-score   support

     Fuzzers       0.99      0.99      0.99     12169
      Normal       1.00      1.00      1.00     46454

    accuracy                           1.00     58623
   macro avg       0.99      0.99      0.99     58623
weighted avg       1.00      1.00      1.00     58623

Test data results
              precision    recall  f1-score   support

     Fuzzers       0.88      0.20      0.33     18184
      Normal       0.79      0.99      0.88     56000

    accuracy                           0.80     74184
   macro avg       0.84      0.60      0.61     74184
weighted avg       0.81      0.80 

## Questions:

1) For results using Part 1 data, what is the precision and recall?

For the Part 1 data, the precision on the test dataset is 0.88 for fuzzers and 0.79 for normal and recall on the test dataset is 0.20 for fuzzers and 0.99 for normal. This means that both normal and fuzzer traffic has approximately the same precision and are more likely to have fewer false positives compared to true positives. The recall values means that normal traffic is predicted correctly most of the time, but there are many false negatives for fuzzer datapoints that are classified as normal.

2) For results using Part 1 data, describe the difference in the results on the training and test data. What does this signify? 

The results of the training and test datasets for the first random forest classifier show that the classifier can accurately and precisely classify the training data, as the values are 1. Compared to the test data, all metrics are smaller than the training metrics to show that the classifier can incorrectly classify the data in the test dataset. This shows that the random forest was correctly trained on the training dataset and has not seen portions of the test dataset.

3) What changes in the results on the test data once you combine the data for Part 2? Does this produce a better classifier? Why or why not?

Yes. Accuracy for the second random forest is higher than the first random forest (0.9 vs 0.8 respectively). The F1 scores for the second random forest is also higher than the first random forest to indicate that the average precision and recall for normal and fuzzers are improved with the second dataset. This means that the test data for the first classifier varies quite a bit compared to the training dataset.